In [1]:
import numpy as np
from matplotlib import pyplot as plt
import keras
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D, BatchNormalization
import os
import random
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from keras import regularizers

Using TensorFlow backend.


In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
!nvidia-smi

Thu Mar 19 09:00:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    25W /  70W |     10MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, batch_size=32, dim=(16, 200), n_channels=1, shuffle=True):
        'Initialization'
        self.files = []
        self.indexes = None
        self.dim = dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.files)/self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        files_temp = [self.files[k] for k in indexes]

        # Generate data
        dat = self._load_files(files_temp)
#         X = np.log(dat + 100)
        X = dat

        return X, X
    
    def load_directory(self, directory, extension="npy"):
        orig_len = len(self.files)
        self.files.extend([os.path.join(directory, x) for x in os.listdir(directory) if x.endswith(extension)])
        self.files = list(set(self.files))
        print("%d files added" % (len(self.files) - orig_len))
        print("%d files in total" % len(self.files))
        if self.shuffle:
            random.shuffle(self.files)
        self.indexes = np.arange(len(self.files))

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            random.shuffle(self.files)

    def _load_files(self, files):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, file in enumerate(files):
            # Store sample
            arr = np.load(file)
#             if arr.shape != (16, 200):
#                 print(arr.shape)
#                 print(file)
            X[i,] = np.expand_dims(arr, axis=2)

        return X

In [5]:
filtered_dir = "../filtered"
data_gen = DataGenerator()

In [6]:
for i in range(22):
    data_gen.load_directory(f"{filtered_dir}/{i}")
# for i in [18, 20]:
#     data_gen.load_directory(f"{filtered_dir}/{i}")

7091 files added
7091 files in total
9 files added
7100 files in total
15 files added
7115 files in total
14 files added
7129 files in total
16 files added
7145 files in total
9959 files added
17104 files in total
506 files added
17610 files in total
49728 files added
67338 files in total
26551 files added
93889 files in total
51170 files added
145059 files in total
61 files added
145120 files in total
3014 files added
148134 files in total
43 files added
148177 files in total
20794 files added
168971 files in total
3195 files added
172166 files in total
68527 files added
240693 files in total
46018 files added
286711 files in total
4992 files added
291703 files in total
47338 files added
339041 files in total
612 files added
339653 files in total
4935 files added
344588 files in total
2588 files added
347176 files in total


In [7]:
input_img = Input(shape=(16, 200, 1))
x = Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
# x = BatchNormalization()(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = Conv2D(32, (3, 3), activation="relu", padding="same")(x)
# x = BatchNormalization()(x)
x = Conv2D(8, (3, 3), activation="relu", padding="same")(x)
# x = MaxPooling2D((2, 2), padding="same")(x)
flat = Flatten()(x)
x = Dense(1024, activation='relu')(flat)
encoded = Dense(784, activation='relu', activity_regularizer=regularizers.l1(10e-10))(x)
x = Dense(1024, activation='relu')(encoded)
x = Dense(3200, activation='relu')(x)
x = Reshape((8, 100, 4))(x)
x = Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
x = Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
x = Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
x = Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
x = Conv2DTranspose(32, (3, 3), activation="relu", padding="same")(x)
# x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), padding="same")(x)

model = Model(inputs=input_img, outputs=decoded)


In [8]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 200, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 200, 32)       320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 200, 32)       9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 200, 32)       9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 100, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 100, 32)        9248      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 100, 32)        9248

In [9]:
encoder = Model(inputs=input_img, outputs=encoded)

In [10]:
class DataShuffleCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        data_gen.on_epoch_end()
    

In [11]:
model.compile("adam", loss="msle")

In [12]:
model.fit_generator(generator=data_gen, epochs=1, callbacks=[DataShuffleCallback()])

Epoch 1/1
 1638/10849 [===>..........................] - ETA: 4:49 - loss: 0.0785

KeyboardInterrupt: 

In [ ]:
test_data = data_gen.__getitem__(2)
test_data[0].shape

In [ ]:
plt.imshow(test_data[0][0, :, :, 0])

In [ ]:
test_data[0].shape

In [ ]:
pred = model.predict(test_data[0])

In [ ]:
encoded_pred = encoder.predict(test_data[0])

In [ ]:
encoded_pred.shape

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
def foo(ind):
    plt.figure()
    plt.title("ground truth")
    plt.imshow(np.squeeze(test_data[0][ind]))
    pred0 = np.squeeze(pred[ind])
    plt.figure()
    plt.title("prediction")
    plt.imshow(pred0)
    
interact(foo,ind=widgets.IntSlider(min=0, max=31, step=1, value=5))

In [ ]:
sample = data_gen.__getitem__(0)[0]
for i in range(200):
    sample = np.append(sample, data_gen.__getitem__(i)[0], axis=0)

In [ ]:
sample.shape

In [ ]:
from sklearn import manifold
import matplotlib.colors as mcolors
color_set = mcolors.TABLEAU_COLORS
cnames = list(color_set.keys())

In [ ]:
encoded_samp = encoder.predict(sample)

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', perplexity=100, n_jobs=-1, verbose=1)
embedded_samp = tsne.fit_transform(encoded_samp)

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib inline

In [ ]:
# 2D
embedded_x, embedded_y = np.split(embedded_samp, 2, axis=1)
plt.scatter(embedded_x, embedded_y)

# # 3D
# fig = plt.figure()
# ax = plt.axes(projection='3d')

# embedded_x, embedded_y, embedded_z = np.split(embedded_samp, 3, axis=1)
# ax.scatter3D(embedded_x, embedded_y, embedded_z)

In [ ]:
inds = np.nonzero((embedded_x > 30) * (embedded_y < 0))[0]

In [ ]:
sample_inds = np.random.choice(inds, size=6)
figure, axes = plt.subplots(nrows=6, ncols=1)
plt.subplot(611)
plt.imshow(sample[sample_inds[0], :, :, 0])
plt.subplot(612)
plt.imshow(sample[sample_inds[1], :, :, 0])
plt.subplot(613)
plt.imshow(sample[sample_inds[2], :, :, 0])
plt.subplot(614)
plt.imshow(sample[sample_inds[3], :, :, 0])
plt.subplot(615)
plt.imshow(sample[sample_inds[4], :, :, 0])
plt.subplot(616)
plt.imshow(sample[sample_inds[5], :, :, 0])
figure.tight_layout(pad=3.0)

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [ ]:
scaled_samp = StandardScaler().fit_transform(encoded_samp)

In [ ]:
db = DBSCAN(eps=0.5, min_samples=6).fit(scaled_samp)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
print(len(labels))

In [ ]:
inds = np.where(labels==1)[0]
print(f"Number of samples: {len(inds)}")

In [ ]:
sample_inds = np.random.choice(inds, size=6)
figure, axes = plt.subplots(nrows=6, ncols=1)
plt.subplot(611)
plt.imshow(sample[sample_inds[0], :, :, 0])
plt.subplot(612)
plt.imshow(sample[sample_inds[1], :, :, 0])
plt.subplot(613)
plt.imshow(sample[sample_inds[2], :, :, 0])
plt.subplot(614)
plt.imshow(sample[sample_inds[3], :, :, 0])
plt.subplot(615)
plt.imshow(sample[sample_inds[4], :, :, 0])
plt.subplot(616)
plt.imshow(sample[sample_inds[5], :, :, 0])
figure.tight_layout(pad=3.0)